# Additional End of week Exercise - week 2

1. Add Another Tool to make a booking. (Just print that a booking is done).
2. Add An Agnet that translates all responses to a different language and shows on the right hand side, using a different fronrier model.
3. Add an agent that can listen for Audio and convert it to Text.

## Initialisations and Definitions 

Here We will do some imports and define the tools we will use for getting a price and booking a ticket.

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
# Acts as a Database
bookings_records = {}

def book_ticket(destination_city, num_tickets, ticket_class, mail_address):
    ticket_price = int(get_ticket_price(destination_city).replace("$", ""))
    total_booking_price = num_tickets * ticket_price
    bookings_records.setdefault(mail_address, []).append({'paid': total_booking_price, 'destination': destination_city, 'number_of_tickets': num_tickets})
    return 'Tickets to ' + destination_city + ' Booked'
    

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [15]:
booking_function = {
        "name": "book_ticket",
        "description": "Book a ticket to a destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city the customer wants to travel to",
                },
                "num_tickets": {
                    "type": "integer",
                    "description": "The number of tickets to book",
                },
                "ticket_class": {
                    "type": "string",
                    "description": "The class of the ticket (e.g., economy, business)",
                },

                "mail_address": {
                    "type": "string",
                    "description": "Mail address to send the ticket to",
                },
            },
            "required": ["destination_city", "num_tickets", "ticket_class", "mail_address"],
            "additionalProperties": False
        }
}

In [16]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": booking_function}]

## Application

In this step, the App is created, communicate with OpenAI, use the two tools we provided, and handle them.

In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message) # Message from Agent
        messages.append(response) # Our response to the message using tools
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [20]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response
    elif tool_call.function.name == "book_ticket":
        arguments = json.loads(tool_call.function.arguments)
        destination_city = arguments.get('destination_city')
        num_tickets = arguments.get('num_tickets')
        ticket_class = arguments.get('ticket_class')
        mail_address = arguments.get('mail_address')
        result = book_ticket(destination_city, num_tickets, ticket_class, mail_address)
        print(result + "\n")
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": destination_city,"num_tickets": num_tickets, "ticket_class": ticket_class}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response

In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tickets to London Booked



In [22]:
print(bookings_records)

{'mail@awesome.com': [{'paid': 1598, 'destination': 'London', 'number_of_tickets': 2}]}


## Real Time Translator

The idea is to translate the chat going on between the user and the chat bot to German on the right hand side.

In [54]:
def translate_message(text, target_language="de"):
    """
    Function to translate the text using GPT model.
    `target_language` specifies the language to translate the text to (default is Spanish).
    """
    prompt = f"Translate the following message to {target_language}: {text}"

    # Making a call to GPT-4 to perform the translation
    response = openai.chat.completions.create(
        model=MODEL,  # You can also use gpt-3.5-turbo or a smaller version
        messages=[
            {"role": "system", "content": "You are a helpful translator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.3
    )

    translated_text = response.choices[0].message.content
    return translated_text

    
def translate_chat(conversation, target_language="de"):
    """
    Translate the entire chat conversation between user and chatbot.
    `conversation` is a list of dictionaries containing 'role' (user or assistant) and 'message'.
    """
    translated_conversation = []

    for message in conversation:
        role = message.get("role")
        original_text = message.get("content")
        
        translated_text = translate_message(original_text, target_language)
        
        translated_conversation.append({
            "role": role,
            "message": translated_text
        })

    return translated_conversation
    


In [57]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    translated_conversation = translate_chat(history, target_language="de")
    print(translated_conversation)
    return history, translation

In [58]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        translation = gr.Textbox()
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, translation]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'message': 'Hallo, wie geht es dir heute?'}, {'role': 'assistant', 'message': 'Ich bin nur ein Programm, aber ich bin hier, um Ihnen zu helfen! Wie kann ich Ihnen heute helfen?'}]
[{'role': 'user', 'message': 'Hallo, wie geht es dir heute?'}, {'role': 'assistant', 'message': 'Ich bin nur ein Programm, aber ich bin hier, um Ihnen zu helfen! Wie kann ich Ihnen heute behilflich sein?'}, {'role': 'user', 'message': 'Ich möchte nach London reisen.'}, {'role': 'assistant', 'message': 'Sicher! Wie viele Tickets benötigen Sie und welche Klasse würden Sie bevorzugen?'}]
[{'role': 'user', 'message': 'Hallo, wie geht es dir heute?'}, {'role': 'assistant', 'message': 'Ich bin nur ein Programm, aber ich bin hier, um Ihnen zu helfen! Wie kann ich Ihnen heute behilflich sein?'}, {'role': 'user', 'message': 'Ich möchte nach London reisen.'}, {'role': 'assistant', 'message': 'Sicher! Wie viele Tickets benötigen Sie und welche Klasse bevorzugen Sie?'}, {'role': 'user', 'message': '2 Ti

Find a way to show the above text in Gradio!!